In [2]:
import sys
import numpy as np
import pickle
import pandas as pd
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import time
import init as x
import update_params as xx
import simulate_corona_single as sc
import simulate_corona_senior_single as scs
import multiprocessing as mp
import save_part as sp

## setting
x.fnc_type = 0 # 0 =  benchmark, 1 = treat senior spec.
x.R0_type = 2  # 0 = normal, 1 = low, 2 = high
xx.set_params()
alpha_min = 0.35
alpha_max = 1
alpha_step = 0.05

## initialize
out_filename_root,out_fig_root,out_stat_root=x.setup_paths(x.fnc_type,x.R0_type)
if x.fnc_type == 0:
    fnc_name = sc.simul
else:
    fnc_name = scs.simul
out_filename = out_filename_root+"/"+x.out_filename

pct_data=[]
raw_data=[]
np.random.seed(123)
alpha_vec = np.arange(alpha_min,alpha_max+alpha_step,alpha_step)
alpha_len = len(alpha_vec)

def run_simulation(fun):
    cores_cnt = mp.cpu_count()
    pool = mp.Pool(cores_cnt)
    tic = time.perf_counter()
    res = pool.map(fun,alpha_vec)
    toc = time.perf_counter()
    pool.close()
    dt_avg = (toc-tic)/x.simul_cnt
    for idx in range(alpha_len):
        ax,bx,alpha_vec[idx] = res[idx]
        pct_data.append(ax)
        raw_data.append(bx)
    return dt_avg

def save_results():
    fid = open(out_filename,'wb')
    pickle.dump([pct_data,raw_data,alpha_vec],fid)
    fid.close()


print("R0:"+str(x.R0))
print("beta:"+str(x.beta))
print("alpha vector:"+str(alpha_vec))

## run all simulations
print("Running simulation ... ",end="",flush=True)
tpi=run_simulation(fnc_name)
print("Finished")
print("Time per iteration (sec.): "+str(tpi))

## save results
print("Saving results ...",end="",flush=True)
save_results()
print("Finished")
    

R0:2.6
beta:0.52
alpha vector:[0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]
Running simulation ... Finished
Time per iteration (sec.): 3.9485844546249993
Saving results ...Finished
